In [1]:
import os

import gmsh
import numpy as np
import pandas as pd

import commons, configs, utils

In [2]:
scaling = configs.get_configs()['CONTACT_LOSS_SCALING']
scale_x = float(scaling['x'])
scale_y = float(scaling['y'])
scale_z = float(scaling['z'])
LX = 470 * scale_x
LY = 470 * scale_y
LZ = 45 * scale_z
LSEP = 15 * scale_z
LCAT = 30 * scale_z
Rp = 6 * scale_z
workdir = "output/reaction_distribution/470-470-45/no-contact-loss"
utils.make_dir_if_missing(workdir)
output_msh_path = os.path.join(workdir, "mesh.msh")
markers = commons.Markers()


In [3]:
df = scale_x * 470 * pd.read_csv('data/laminate.csv')

In [4]:
cylinders = []
gmsh.initialize()
gmsh.model.add('geo')
box_left = gmsh.model.occ.addBox(0, 0, 0, LX, LY, LZ)
box_right = gmsh.model.occ.addBox(0, 0, LZ - Rp, LX, LY, Rp)
for idx in range(df.shape[0]):
    x, y = df.loc[idx, :]
    if (x + Rp) >= LX or (y + Rp) >= LY:
        continue
    if (x - Rp) <= 0 or (y - Rp) <= 0:
        continue
    cyl = gmsh.model.occ.addCylinder(x, y, LSEP, 0, 0, LCAT - Rp, Rp)
    cylinders.append((3, cyl))
merged = gmsh.model.occ.fuse([(3, box_right)], cylinders)
gmsh.model.occ.cut([(3, box_left)], merged[0], removeTool=False)

gmsh.model.occ.synchronize()
vols = gmsh.model.get_entities(3)
gmsh.model.addPhysicalGroup(3, [vols[0][1]], markers.electrolyte, "electrolyte")
gmsh.model.addPhysicalGroup(3, [vols[1][1]], markers.positive_am, "positive_am")

surfs = gmsh.model.get_entities(2)
left = []
right = []
interface = []
insulated_se = []
insulated_am = []
for _, surf in surfs:
    com = gmsh.model.occ.get_center_of_mass(2, surf)
    if np.isclose(com[2], 0):
        left.append(surf)
    elif np.isclose(com[2], LZ):
        right.append(surf)
    elif np.isclose(com[2], 0.5*(LZ - Rp)):
        insulated_se.append(surf)
    elif np.isclose(com[2], LZ - 0.5 * Rp):
        insulated_am.append(surf)
    else:
        interface.append(surf)
gmsh.model.addPhysicalGroup(2, left, markers.left, "left")
gmsh.model.addPhysicalGroup(2, insulated_se, markers.insulated_electrolyte, "insulated_electrolyte")
gmsh.model.addPhysicalGroup(2, right, markers.right, "right")
gmsh.model.addPhysicalGroup(2, insulated_am, markers.insulated_positive_am, "insulated_am")
gmsh.model.addPhysicalGroup(2, interface, markers.electrolyte_v_positive_am, "electrolyte_positive_am_interface")
gmsh.model.occ.synchronize()
gmsh.model.mesh.generate(3)
gmsh.write(output_msh_path)
gmsh.finalize()

Info    : Meshing 1D...
Info    : [  0%] Meshing curve 13 (Line)
Info    : [ 10%] Meshing curve 14 (Line)
Info    : [ 10%] Meshing curve 15 (Line)
Info    : [ 10%] Meshing curve 16 (Line)
Info    : [ 10%] Meshing curve 17 (Line)
Info    : [ 10%] Meshing curve 18 (Line)
Info    : [ 10%] Meshing curve 19 (Line)
Info    : [ 10%] Meshing curve 20 (Line)
Info    : [ 10%] Meshing curve 21 (Line)
Info    : [ 10%] Meshing curve 22 (Line)
Info    : [ 10%] Meshing curve 23 (Line)
Info    : [ 10%] Meshing curve 24 (Line)
Info    : [ 10%] Meshing curve 25 (Circle)
Info    : [ 10%] Meshing curve 26 (Circle)
Info    : [ 10%] Meshing curve 27 (Circle)
Info    : [ 10%] Meshing curve 28 (Circle)
Info    : [ 10%] Meshing curve 29 (Circle)
Info    : [ 10%] Meshing curve 30 (Circle)
Info    : [ 10%] Meshing curve 31 (Circle)
Info    : [ 10%] Meshing curve 32 (Circle)
Info    : [ 10%] Meshing curve 33 (Circle)
Info    : [ 10%] Meshing curve 34 (Circle)
Info    : [ 10%] Meshing curve 35 (Circle)
Info    : [

Info    : 1289 edge swaps, 290 node relocations (volume = 8.79609e-12): worst = 0.000701423 / average = 0.395887 (Wall 0.189048s, CPU 0.189926s)
Info    : 1804 edge swaps, 473 node relocations (volume = 8.79609e-12): worst = 0.000701423 / average = 0.394731 (Wall 0.378015s, CPU 0.379133s)
Info    : 0.00 < quality < 0.10 :     14511 elements
Info    : 0.10 < quality < 0.20 :      2251 elements
Info    : 0.20 < quality < 0.30 :      1110 elements
Info    : 0.30 < quality < 0.40 :      1559 elements
Info    : 0.40 < quality < 0.50 :      1137 elements
Info    : 0.50 < quality < 0.60 :      1542 elements
Info    : 0.60 < quality < 0.70 :      2592 elements
Info    : 0.70 < quality < 0.80 :      2432 elements
Info    : 0.80 < quality < 0.90 :      6184 elements
Info    : 0.90 < quality < 1.00 :      2767 elements
Info    : Done optimizing mesh (Wall 0.750279s, CPU 0.752925s)
Info    : 22805 nodes 163150 elements
Info    : Writing 'output/reaction_distribution/40-40-150/mesh.msh'...
Info    